In [ ]:
import numpy as np
import xarray as xr
import pickle
import pandas as pd
import time
import os

import datetime
from sys import getsizeof,path


import matplotlib.pyplot as plt
import matplotlib

import theano
import theano.tensor as tt
import pymc3 as pm
theano.config.optimizer="fast_run"

In [ ]:
class TimeSplitLoop(object):
    def __init__(self,major=None,minor=None,scaling=7,other_dims=0):
        if major == None:
            major = (3,52)
        if minor == None:
            minor = (3,10)
        
        self.major = major
        self.minor = minor
        self.scaling = scaling
        
        self.other_dims = other_dims
        
    def initial(self,dtype="float64"):
        shape = []
        shape += [self.major[0]*self.scaling+self.major[1]]
        shape += [self.minor[0]*self.scaling+self.minor[1]]
        shape += [1]*self.other_dims

        return tt.ones(shape,dtype)
    
    def run_fine_unrolled(self,n=10):
        values = tt.cast([1.01]*n,"float64")
        
        def fn(value,state):
            acc = tt.zeros(self.minor[0]*self.scaling+self.minor[1],"float64")
            for i in range(self.scaling):
                
                # Shift fine-grained by 1, accumulate value from last fine-grained-cell 
                acc += state[self.major[0]*self.scaling-1]
                state = tt.set_subtensor(state[1:self.major[0]*self.scaling],state[:self.major[0]*self.scaling-1])
                state = tt.set_subtensor(state[0],state[0]*value)
                
            # Shift rough-grained by 1, inject fine-accumulated at the beginning.
            state = tt.set_subtensor(state[self.major[0]*self.scaling+1:],state[self.major[0]*self.scaling:-1])
            state = tt.set_subtensor(state[self.major[0]*self.scaling],acc)

            #            state = tt.set_subtensor(state[self.major[0]*self.scaling],tt.sum(state[(self.major[0]-1)*self.scaling:self.major[0]*self.scaling],axis=0))
            return state
        
        states,end = theano.scan(fn,outputs_info=[self.initial()],sequences=[values])
                
        return states
        

In [ ]:
tsl = TimeSplitLoop((2,6,),(2,4,))

print(tsl.initial().eval().shape)


t0 = time.time()
result = tsl.run_fine_unrolled()
states = result[-1].eval()
t1 = time.time()

print(t1-t0)

#print(states)

theano.printing.debugprint(result)
theano.printing.debugprint(states)